# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

* Litesh

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
# filepath 
file = "../output/cities.csv"

# read the csv file into dataframe
city_weather_df = pd.read_csv(file)

#display df
city_weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mataura,-46.1927,168.8643,45.48,90,10,3.31,NZ,1624080064
1,punta arenas,-53.1500,-70.9167,37.51,65,0,4.61,CL,1624080029
2,upernavik,72.7868,-56.1549,33.49,72,3,5.37,GL,1624080066
3,port alfred,-33.5906,26.8910,56.08,80,6,8.99,ZA,1624080067
4,santiago,-33.4569,-70.6483,52.86,73,90,2.30,CL,1624079770
...,...,...,...,...,...,...,...,...,...
567,cairns,-16.9167,145.7667,86.95,51,28,5.75,AU,1624080540
568,pandan,14.0453,124.1694,90.48,58,57,5.35,PH,1624080541
569,biak,-0.9131,122.8766,82.11,72,74,4.00,ID,1624080542
570,north bend,43.4065,-124.2243,55.92,86,75,24.16,US,1624080542


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Get locations and humidity
locations = city_weather_df[["Lat", "Lng"]]
weights = city_weather_df["Humidity"]

# Format heatmap
layout = {
    'width': "100%",
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',

}
#Formattting
fig = gmaps.figure(layout=layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100,
                                 point_radius = 1)
#Add the heatmap layer
fig.add_layer(heatmap_layer)

#display heatmap
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# Dataframe to hold ideal weather conditions
ideal_cities = city_weather_df.loc[(city_weather_df["Max Temp"] > 70) & 
                               (city_weather_df["Max Temp"] < 80) & 
                               (city_weather_df["Wind Speed"] < 10) & 
                               (city_weather_df["Cloudiness"] == 0) &
                               #drop NA
                               (city_weather_df["Humidity"] < 75), :].reset_index(drop=True).dropna()
ideal_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sakakah,29.9697,40.2064,78.37,21,0,8.01,SA,1624080070
1,ardud,47.6333,22.8833,73.29,64,0,4.61,RO,1624080089
2,kangavar,34.5043,47.9653,76.48,14,0,4.47,IR,1624080121
3,carnarvon,-24.8667,113.6333,73.47,37,0,3.44,AU,1624080224
4,nikolskoye,59.7035,30.7861,76.89,54,0,7.14,RU,1624080236
5,port hedland,-20.3167,118.5667,77.05,44,0,9.22,AU,1624080245
6,suez,29.9737,32.5263,71.24,70,0,4.52,EG,1624080276
7,khvoynaya,58.9000,34.5333,71.69,57,0,8.10,RU,1624080297
8,waddan,29.1614,16.1390,74.86,39,0,4.72,LY,1624080382
9,damietta,31.4165,31.8133,75.09,64,0,2.84,EG,1624080432


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# Dataframe hotel_df 
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng"]].copy()

# --- Add a "Hotel Name" column to the DataFrame with null values ---


hotel_df["Hotel Name"] = np.nan
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,sakakah,SA,29.9697,40.2064,NaN
1,ardud,RO,47.6333,22.8833,NaN
2,kangavar,IR,34.5043,47.9653,NaN
3,carnarvon,AU,-24.8667,113.6333,NaN
4,nikolskoye,RU,59.7035,30.7861,NaN
5,port hedland,AU,-20.3167,118.5667,NaN
6,suez,EG,29.9737,32.5263,NaN
7,khvoynaya,RU,58.9000,34.5333,NaN
8,waddan,LY,29.1614,16.1390,NaN
9,damietta,EG,31.4165,31.8133,NaN


In [8]:
ideal_cities

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,29.9697,40.2064,78.37,21,0,8.01,SA,1624080070
1,47.6333,22.8833,73.29,64,0,4.61,RO,1624080089
2,34.5043,47.9653,76.48,14,0,4.47,IR,1624080121
3,-24.8667,113.6333,73.47,37,0,3.44,AU,1624080224
4,59.7035,30.7861,76.89,54,0,7.14,RU,1624080236
5,-20.3167,118.5667,77.05,44,0,9.22,AU,1624080245
6,29.9737,32.5263,71.24,70,0,4.52,EG,1624080276
7,58.9000,34.5333,71.69,57,0,8.10,RU,1624080297
8,29.1614,16.1390,74.86,39,0,4.72,LY,1624080382
9,31.4165,31.8133,75.09,64,0,2.84,EG,1624080432


In [17]:
# --- set up parameters for querying Google Places API ---
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# --- set up a parameters dictionary ---
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# loop to go through each city for an API call 

for index, row in hotel_df.iterrows():
    
    # coordinates and store in variables
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # API call, hotel details for each city 
    hotel_data = requests.get(base_url, params).json()
    
    # save the first hotel name for each city
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"City '{row['City']}' found! The nearest hotel is {hotel_name}")
        print("-------------------------------------------------------------------------")
              
    except:
        print(f"Could not find information for city '{row['City']}'... skipping.")    
        print("-------------------------------------------------------------------------")

City 'sakakah' found! The nearest hotel is Alnuzl Special Hotel
-------------------------------------------------------------------------
City 'ardud' found! The nearest hotel is Pensiunea Primăverii
-------------------------------------------------------------------------
City 'carnarvon' found! The nearest hotel is Carnarvon Motel
-------------------------------------------------------------------------
Could not find information for city 'nikolskoye'... skipping.
-------------------------------------------------------------------------
City 'port hedland' found! The nearest hotel is Hedland Hotel
-------------------------------------------------------------------------
City 'suez' found! The nearest hotel is نادى البحارة الدولى بالسويس - International Seamen's Club Suez
-------------------------------------------------------------------------
City 'khvoynaya' found! The nearest hotel is Gostinitsa Khvoynaya
-------------------------------------------------------------------------
Ci

In [14]:
#dropy na values
hotel_df.dropna(how="any",inplace=True)
hotel_df.reset_index(drop=True, inplace=True)

#display df
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,sakakah,SA,29.9697,40.2064,Alnuzl Special Hotel
1,ardud,RO,47.6333,22.8833,Pensiunea Primăverii
2,carnarvon,AU,-24.8667,113.6333,Carnarvon Motel
3,nikolskoye,RU,59.7035,30.7861,Tourist House - Sablino
4,port hedland,AU,-20.3167,118.5667,Hedland Hotel
5,suez,EG,29.9737,32.5263,نادى البحارة الدولى بالسويس - International Se...
6,khvoynaya,RU,58.9000,34.5333,Gostinitsa Khvoynaya
7,damietta,EG,31.4165,31.8133,SULIMAN INN


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
